## 多因子策略（二）

In [ ]:
#========================================================================================
# 多因子策略（二）
from datetime import timedelta, date
import pandas as pd
#导包操作，我们需要分析时间和数据的两个工具pandas、datetime
#========================================================================================
#========================================================================================
#初始化函数，我们设置最大持股15个，设置交易频率为1-12月，设置trade函数每个月月末运行一次。
def init(context):
    context.n = 15 
    run_monthly(trade,date_rule=-1)
#========================================================================================
#========================================================================================
#设置交易函数，首先判断时间月份是够满足，随后获取选股函数的结果，将不符合的卖出后符合的买入。
def trade(context, bar_dict):
    pe_ttm_list = stocks_pe_ttm(context,bar_dict)
    netProfitGrowthrate_list=stocks_netProfitGrowthrate(context,bar_dict)
    ps_ttm_list=stocks_ps_ttm(context,bar_dict)
    stock_list = list(set(pe_ttm_list)&set(ps_ttm_list)&set(netProfitGrowthrate_list))
    #这条代码的&是且的意思，也就是三个set()全部满足的股票构成一个列表。
    if len(list(context.portfolio.stock_account.positions.keys())) > 0:
        for stock in list(context.portfolio.stock_account.positions.keys()):
            if stock not in stock_list:
                order_target(stock, 0)
    if len(stock_list) > 0:
        for stock in stock_list:
            if stock not in list(context.portfolio.stock_account.positions.keys()):
                if len(list(context.portfolio.stock_account.positions.keys())) < context.n :
                    number = context.n  - len(list(context.portfolio.stock_account.positions.keys()))
                    order_value(stock,context.portfolio.stock_account.available_cash/number)
                else: 
                    order_value(stock,context.portfolio.stock_account.available_cash)
#========================================================================================
#========================================================================================
#以下三个是自定义函数区，第一个是根据pe_ttm选股，第二个是净利润增长率，第三个是市销率。
#三个选股函数性质一致，主要是靠62行代码将其整合，只有会整合，你可以设置很多选股函数，然后自由组合。
def stocks_pe_ttm(context,bar_dict):
    last_date = get_last_datetime().strftime('%Y%m%d')
    pe_ttm = get_fundamentals(query(
            valuation.symbol,
            valuation.pe_ttm
        ).filter(
            valuation.pe_ttm > 0,
        ).order_by(
            valuation.pe_ttm.asc()
        ),date = last_date)
    #get_fundamentals是获取财务数据函数，query是对象，filter是筛选条件，order_by是排序，其中asc()是正序，
    #desc()是逆序
    n = len(pe_ttm)
    #n是获取选股结果的数量
    return list(pe_ttm['valuation_symbol'][:int(n*0.2)])
    #int()是转成成数字格式
    #return是输出函数结果，这里是前20%的个股
def stocks_netProfitGrowthrate(context,bar_dict):
    last_date = get_last_datetime().strftime('%Y%m%d')
    net_profit_growth_ratio = get_fundamentals(query(
                                  growth.symbol,
                                  growth.net_profit_growth_ratio 
                                  ).filter(growth.net_profit_growth_ratio>0
                                  ).order_by(growth.net_profit_growth_ratio.desc()
                                  ),date = last_date)
    n = len(net_profit_growth_ratio)
    return list(net_profit_growth_ratio['growth_symbol'][:int(n*0.2)])
def stocks_ps_ttm(context,bar_dict):
    last_date = get_last_datetime().strftime('%Y%m%d')
    ps_ttm = get_fundamentals(query(
            valuation.symbol,
            valuation.ps_ttm
        ).filter(
            valuation.ps_ttm>0
        ).order_by(
            valuation.ps_ttm.asc()      
        ),date = last_date)
    n=len(ps_ttm)
    return list(ps_ttm['valuation_symbol'][:int(n*0.2)])
#========================================================================================
#========================================================================================

